In [13]:
import numpy as np
import random

In [3]:
import pandas as pd

In [64]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

In [65]:
word_vectors = KeyedVectors.load_word2vec_format('../data/embedding/rdf2vec_vectors.txt', binary=False)

# C text format

In [66]:
word_vectors.most_similar('https://foodb.ca/foods/4', topn=5)

[('https://foodb.ca/foods/foodName/Kiwi', 0.8686439990997314),
 ('https://foodb.ca/foods/790', 0.8090094327926636),
 ('https://foodb.ca/foods/229', 0.7959798574447632),
 ('https://foodb.ca/foods/81', 0.7850533723831177),
 ('https://foodb.ca/foods/256', 0.7843894362449646)]

In [72]:
df= pd.read_csv('../data/nutri_scores.csv')
df.head()

,Unnamed: 0,food_id,food_name,food_group,food_subgroup,source_name,nutri_score,nutri_score_category,hasQuality
0,0,4,Kiwi,Fruits,Tropical fruits,Fat,-1,A,"['high_vitaminK1', 'high_vitaminB5']"
1,1,6,Garden onion,Vegetables,Onion-family vegetables,Fat,-4,A,"['high_potassium', 'high_iron', 'high_phosphor..."
2,2,9,Chives,Herbs and Spices,Herbs,Fat,-3,A,"['high_vitaminK1', 'high_vitaminB5', 'high_vit..."
3,3,11,Cashew nut,Nuts,Nuts,Fat,14,D,"['high_iron', 'high_phosphorus', 'high_vitamin..."
4,4,12,Pineapple,Fruits,Tropical fruits,Fat,2,B,"['high_vitaminB5', 'high_vitaminB8']"


In [74]:
df= df[['food_id', 'nutri_score']]

In [83]:
df['food_id']= df['food_id'].astype(str)

In [85]:
df['food_id']= 'https://foodb.ca/foods/'+df['food_id']

In [86]:
nutriscore ={}
for i, row in df.iterrows():
    nutriscore[row['food_id']] = row['nutri_score']

In [88]:
list(nutriscore.items())[:10]

[('https://foodb.ca/foods/4', -1),
 ('https://foodb.ca/foods/6', -4),
 ('https://foodb.ca/foods/9', -3),
 ('https://foodb.ca/foods/11', 14),
 ('https://foodb.ca/foods/12', 2),
 ('https://foodb.ca/foods/13', 7),
 ('https://foodb.ca/foods/14', 3),
 ('https://foodb.ca/foods/15', -1),
 ('https://foodb.ca/foods/16', 7),
 ('https://foodb.ca/foods/17', 0)]

In [69]:
def get_simscostringrank_onlyss_multisamerank(fromt, asdf):
    # return rank based on KG embedding +revised nutriscore
    mostSimilar =word_vectors.most_similar(fromt, topn=100)
    for i,(e, sim) in enumerate(mostSimilar):
        if e == asdf:
            return i+1    
    return 100

In [96]:
def get_simscore_ingrank_kg_nutriscore_multisamerank(fromt, asdf):
    # return rank based on KG embedding +revised nutriscore
    mostSimilar =word_vectors.most_similar(fromt, topn=20)
    nutriscore_ranking= [ (e,nutriscore[e]) for (e, sim) in mostSimilar if e in nutriscore]
    nutriscore_ranking= sorted(nutriscore_ranking, key= lambda x:x[1])
    #print (nutriscore_ranking)
    for i,(e, sim) in enumerate(nutriscore_ranking):
        if e == asdf:
            return i+1    
    return 20

In [90]:
def mrr_map_new(scraped_subs_dict):
    rank_scores = []
    ave_p = []
    in_top_5 = 0
    in_top_10 = 0
    print('number of ings: ',len(scraped_subs_dict.keys()))

    for fromt in scraped_subs_dict.keys():
        relevant_ranks = []
        min_rank = 9999999999999999
        for asdf in scraped_subs_dict[fromt]:
            #rank = get_simscore_ingrank_onlyss_multisamerank(fromt, asdf)
            rank = get_simscore_ingrank_kg_nutriscore_multisamerank(fromt, asdf)   
            relevant_ranks.append(rank)
            if fromt == 'potato':
                print(asdf, rank)
            if rank < min_rank:
                min_rank = rank
        rank = min_rank

        rank_scores.append(1.0/rank)
        if min_rank <= 5:
            in_top_5 += 1
        if min_rank <= 10:
            in_top_10 += 1

        precisions = []
        for rank in relevant_ranks:
            good_docs = len([r for r in relevant_ranks if r <= rank])
            precisions.append(good_docs/rank)
        if len(precisions) == 0:
            precisions = [0]
        ave_p.append(np.mean(precisions))

    print('mean reciprocal rank: ', np.mean(rank_scores))
    print('recallrate at 5: ', in_top_5/len(scraped_subs_dict.keys()))
    print('recallrate at 10: ', in_top_10/len(scraped_subs_dict.keys()))
    print('mean average precision: ', np.mean(ave_p))
    print('formatted')
    print(round(np.mean(ave_p), 3),' & ',round(np.mean(rank_scores), 3),' & ',round(in_top_5/len(scraped_subs_dict.keys()), 3),' & ',round(in_top_10/len(scraped_subs_dict.keys()), 3))
    return round(np.mean(ave_p), 3)


In [5]:
df = pd.read_json('../data/foodcom_review_data.json')

In [6]:
df.head()

,0,1
0,http://idea.rpi.edu/heals/kb/ingredientname/mo...,http://idea.rpi.edu/heals/kb/ingredientname/ag...
1,http://idea.rpi.edu/heals/kb/ingredientname/ta...,http://idea.rpi.edu/heals/kb/ingredientname/fe...
2,http://idea.rpi.edu/heals/kb/ingredientname/pl...,http://idea.rpi.edu/heals/kb/ingredientname/mi...
3,http://idea.rpi.edu/heals/kb/ingredientname/ma...,http://idea.rpi.edu/heals/kb/ingredientname/oil
4,http://idea.rpi.edu/heals/kb/ingredientname/ca...,http://idea.rpi.edu/heals/kb/ingredientname/sw...


In [8]:
scraped_subs_dict = dict()
for i,row in df.iterrows():
    if row[0] not in scraped_subs_dict:
        scraped_subs_dict[row[0]] = set()
    scraped_subs_dict[row[0]].add(row[1])

In [12]:
list(scraped_subs_dict.items())[:5]

[('http://idea.rpi.edu/heals/kb/ingredientname/molasses',
  {'http://idea.rpi.edu/heals/kb/ingredientname/agave%20nectar',
   'http://idea.rpi.edu/heals/kb/ingredientname/banana',
   'http://idea.rpi.edu/heals/kb/ingredientname/brown%20sugar',
   'http://idea.rpi.edu/heals/kb/ingredientname/corn%20syrup',
   'http://idea.rpi.edu/heals/kb/ingredientname/dark%20brown%20sugar',
   'http://idea.rpi.edu/heals/kb/ingredientname/golden%20syrup',
   'http://idea.rpi.edu/heals/kb/ingredientname/honey',
   'http://idea.rpi.edu/heals/kb/ingredientname/karo%20syrup',
   'http://idea.rpi.edu/heals/kb/ingredientname/maple%20syrup',
   'http://idea.rpi.edu/heals/kb/ingredientname/pure%20maple%20syrup',
   'http://idea.rpi.edu/heals/kb/ingredientname/sorghum',
   'http://idea.rpi.edu/heals/kb/ingredientname/treacle',
   'http://idea.rpi.edu/heals/kb/ingredientname/white%20sugar'}),
 ('http://idea.rpi.edu/heals/kb/ingredientname/tarragon',
  {'http://idea.rpi.edu/heals/kb/ingredientname/basil',
   'htt

In [17]:
mrr_map_new(scraped_subs_dict)

number of ings:  1041
mean reciprocal rank:  0.45740512022932767
recallrate at 5:  0.6878001921229587
recallrate at 10:  1.0
mean average precision:  0.5744500596083637
formatted
0.574  &  0.457  &  0.688  &  1.0


0.574

In [38]:
linking_df = pd.read_csv('../data/limes_accepted_name.nt.txt', names=['entity1', 'predicate', 'entity2','x'], sep=' ')

In [57]:
mapping={}
for i, row in linking_df.iterrows():
    entity1= row['entity1'].replace("<","")[:-1]
    entity2= row['entity2'].replace("<","")[:-1]
    #print (entity1, entity2)
    mapping[entity1] = entity2

In [59]:
scraped_subs_dict = dict()
for i,row in df.iterrows():
    if row[0] in mapping and row[1] in mapping: 
        food= mapping[row[0]]
        subs = mapping[row[1]]
        if food not in scraped_subs_dict:
            scraped_subs_dict[food] = set()
        scraped_subs_dict[food].add(subs)

In [70]:
mrr_map_new(scraped_subs_dict)

number of ings:  130
mean reciprocal rank:  0.13385401046071504
recallrate at 5:  0.2153846153846154
recallrate at 10:  0.3076923076923077
mean average precision:  0.08348331292184108
formatted
0.083  &  0.134  &  0.215  &  0.308


0.083

In [97]:
mrr_map_new(scraped_subs_dict)

number of ings:  130
mean reciprocal rank:  0.10177078577531067
recallrate at 5:  0.13076923076923078
recallrate at 10:  0.2076923076923077
mean average precision:  0.15773695927078818
formatted
0.158  &  0.102  &  0.131  &  0.208


0.158